# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [59]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from skimage.transform import resize #Library to resize images to a target size.

We set the random seed so that the results don't vary drastically.

In [60]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [61]:
train_doc = np.random.permutation(open('C:\\Users\\varma\\Downloads\\Project_data\\Project_data\\train.csv').readlines())
val_doc = np.random.permutation(open('C:\\Users\\varma\\Downloads\\Project_data\\Project_data\\val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [62]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(0,30)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = round(len(folder_list) / batch_size)
        for batch in range(num_batches):
            #print("batch:",batch)# we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #code to resize the images
                    image = imresize(image, (100,100))
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0])
                    batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1])
                    batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            
        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [63]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:\\Users\\varma\\Downloads\\Project_data\\Project_data\\train'
val_path = 'C:\\Users\\varma\\Downloads\\Project_data\\Project_data\\val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 3
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 3


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [58]:
from keras.models import Sequential, Model
from keras.layers import Dense,LSTM, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

input_shape = (30,100, 100, 3)
num_classes = 5
#Conv3D model architecture
CNNLSTM_Model = Sequential()


# define CNN model
CNNLSTM_Model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')
                                  ,input_shape=input_shape,name = "TimeDist_Conv2D_32")) 

CNNLSTM_Model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),name = "TimeDist_Conv2D_64"))
CNNLSTM_Model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)),name = "TimeDist_MaxPool"))

CNNLSTM_Model.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),name = "TimeDist_Conv2D_128"))
CNNLSTM_Model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)),name = "TimeDist_MaxPool_2"))

CNNLSTM_Model.add(TimeDistributed(Flatten(),name = "TimeDist_Flatten"))

# define LSTM model
CNNLSTM_Model.add(LSTM(128, return_sequences=True, name = "LSTM_128"))

CNNLSTM_Model.add(LSTM(64, return_sequences=False, name = "LSTM_64_2"))

CNNLSTM_Model.add(Dense(num_classes, activation='softmax'))

print(CNNLSTM_Model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
TimeDist_Conv2D_32 (TimeDist (None, 30, 98, 98, 32)    896       
_________________________________________________________________
TimeDist_Conv2D_64 (TimeDist (None, 30, 96, 96, 64)    18496     
_________________________________________________________________
TimeDist_MaxPool (TimeDistri (None, 30, 48, 48, 64)    0         
_________________________________________________________________
TimeDist_Conv2D_128 (TimeDis (None, 30, 46, 46, 128)   73856     
_________________________________________________________________
TimeDist_MaxPool_2 (TimeDist (None, 30, 23, 23, 128)   0         
_________________________________________________________________
TimeDist_Flatten (TimeDistri (None, 30, 67712)         0         
_________________________________________________________________
LSTM_128 (LSTM)              (None, 30, 128)           34734592  
__________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [65]:
optimiser = optimizers.SGD(0.01)
CNNLSTM_Model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (CNNLSTM_Model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
TimeDist_Conv2D_32 (TimeDist (None, 30, 98, 98, 32)    896       
_________________________________________________________________
TimeDist_Conv2D_64 (TimeDist (None, 30, 96, 96, 64)    18496     
_________________________________________________________________
TimeDist_MaxPool (TimeDistri (None, 30, 48, 48, 64)    0         
_________________________________________________________________
TimeDist_Conv2D_128 (TimeDis (None, 30, 46, 46, 128)   73856     
_________________________________________________________________
TimeDist_MaxPool_2 (TimeDist (None, 30, 23, 23, 128)   0         
_________________________________________________________________
TimeDist_Flatten (TimeDistri (None, 30, 67712)         0         
_________________________________________________________________
LSTM_128 (LSTM)              (None, 30, 128)           34734592  
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [66]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [67]:
model_name = 'CNNLSTM_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(patience=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [68]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [69]:
CNNLSTM_Model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, 
                     validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\varma\Downloads\Project_data\Project_data\val ; batch size = 10
Source path = Epoch 1/3 C:\Users\varma\Downloads\Project_data\Project_data\train ; batch size = 10



C:\Anaconda\lib\site-packages\ipykernel\__main__.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 145s 2s/step - loss: 1.6212 - categorical_accuracy: 0.2060 - val_loss: 1.6065 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to CNNLSTM_model_init_2018-10-1901_42_56.409139/model-00001-1.62118-0.20597-1.60647-0.21000.h5
Epoch 2/3
67/67 [==============================] - 140s 2s/step - loss: 1.6218 - categorical_accuracy: 0.1821 - val_loss: 1.6083 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to CNNLSTM_model_init_2018-10-1901_42_56.409139/model-00002-1.62175-0.18209-1.60827-0.21000.h5
Epoch 3/3
67/67 [==============================] - 143s 2s/step - loss: 1.6107 - categorical_accuracy: 0.1866 - val_loss: 1.6083 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to CNNLSTM_model_init_2018-10-1901_42_56.409139/model-00003-1.61074-0.18657-1.60826-0.21000.h5
